<a href="https://colab.research.google.com/github/nguyen-nhat-mai/object_detection/blob/model_maxime/ModelCRPv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google.colab
from google.colab import drive
drive = drive.mount('/content/drive')

In [ ]:
pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt  # install dependencies

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
cvxpy 1.2.3 requires setuptools<=64.0.2, but you have setuptools 67.4.0 which is incompatible.


In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline
import os, re, sys, random, shutil, cv2
from datetime import datetime

import torch
from torchvision import models
from torch import nn
from torchvision import transforms
import torch.nn.functional as F
import torch.nn as nn
import torchsummary

from utils.loss import ComputeLoss

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, classes = 14) #channels = 1) we do not need to chose it. it automatically rescales grey to rgb

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-3-2 Python-3.8.10 torch-1.13.1+cu116 CPU

Overriding model.yaml nc=80 with nc=14

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  

In [ ]:
#Learn how to use these parameters
model.conf = 0.25  # NMS confidence threshold
model.iou = 0.45  # NMS IoU threshold
model.agnostic = False  # NMS class-agnostic
model.multi_label = False  # NMS multiple labels per box
model.classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for COCO persons, cats and dogs
model.max_det = 10  # maximum number of detections per image
model.amp = False  # Automatic Mixed Precision (AMP) inference


In [ ]:
# Choose the hyperparameters for training: 
num_epochs = 10
batch_size = 16
lr = 1e-3

# Use mean squared loss function 
criterion = ComputeLoss(model = model)

# It is initialized on our model
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

net_args = {
    "dataset": "LabeledAirportMarch2023",
    "learning_rate": lr,
    "optimizer": "SGD"
            }

In [ ]:
#To see what layers we have
for layer in model.children():
  print(layer)

#can also use list(model.modules())

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device used: {device}')

Device used: cpu


In [ ]:
#Use of wandb to follow the changes
!pip install wandb -qU
import wandb
def wandb_connect():
    wandb_api_key_label = "wandb_api_key"
    wandb_api_key = "9855d20a7918a7c88f92b87c467fb3cc36101f41" # API key from WandB interface

    wandb_conx = wandb.login(key = wandb_api_key)
    print(f"Connected to Wandb online interface : {wandb_conx}")

wandb_connect()

In [ ]:
def training_loop(model, epochs, train_dataloader, val_dataloader, optimizer, criterion, device = "cpu"):
    total_train_losses = []
    total_val_losses = []
    model.to(device)

    for epoch in tqdm(range(1,epochs+1)):
        #print(f'epoch: {epoch}')
        ##TRAINING##
        model.train()
        train_losses = []
        trn_lbl = torch.Tensor([])
        trn_preds = torch.Tensor([])
        for i, batch, in enumerate(train_dataloader):
            img_batch, lbl_batch = batch
            trn_lbl=torch.cat((trn_lbl, lbl_batch))
            img_batch, lbl_batch = img_batch.to(device), lbl_batch.to(device)

            optimizer.zero_grad()
            logits = model(img_batch).to(device)
            trn_preds=torch.cat((trn_preds, logits.argmax(1).cpu()))

                        
            # One-hot encoding or labels so as to calculate MSE error:
            labels_one_hot = torch.FloatTensor(lbl_batch.shape[0], 10).to(device)
            labels_one_hot.zero_()
            labels_one_hot.scatter_(1, lbl_batch.view(-1, 1), 1)

            loss=criterion(logits, labels_one_hot)
            wandb.log({"train_loss":loss.item()}) # log the training loss at each batch
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())

        train_loss_mean = np.mean(train_losses)
        total_train_losses.append(train_loss_mean)

        ##VALIDATION##
        model.eval()
        val_losses = []
        val_lbl = torch.Tensor([])
        val_preds = torch.Tensor([])
        with torch.no_grad():

            for i, batch, in enumerate(val_dataloader):
                img_batch, lbl_batch = batch
                val_lbl=torch.cat((val_lbl, lbl_batch))
                img_batch, lbl_batch = img_batch.to(device), lbl_batch.to(device)


                # One-hot encoding or labels so as to calculate MSE error:

                labels_one_hot = torch.FloatTensor(lbl_batch.shape[0], 10).to(device)
                labels_one_hot.zero_()
                labels_one_hot.scatter_(1, lbl_batch.view(-1, 1), 1)

                logits=model(img_batch).to(device)

                val_preds=torch.cat((val_preds, logits.argmax(1).cpu()))
                loss=criterion(logits, labels_one_hot)

                val_losses.append(loss.item())
                wandb.log({"val_loss":loss.item()})

        val_acc=accuracy(val_preds, val_lbl)
        val_loss_mean = np.mean(val_losses)
        wandb.log({"train_acc":train_acc, "val_acc":val_acc}) # log the train & val accuracy and the val loss at each epoch
        total_val_losses.append(val_loss_mean)